# Integration Health Index (IHI) Trajectory Analysis

This notebook demonstrates Integration Health Index (IHI) computation and longitudinal trajectory visualization for detecting organizational silos.

## IHI Equation (Equation 5)

$$IHI = \frac{1}{2} \left[ \frac{1}{N_p}\sum P_I + \frac{1}{N_r}\sum R_I \right]$$

**Interpretation**:
- **IHI > 0.75**: Strong cross-category integration
- **IHI 0.60-0.75**: Moderate integration
- **IHI < 0.60**: Weak integration (siloed operations)

In [ ]:
import sys
sys.path.append('../src')

from adli_letci_core import compute_integration_health_index

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("IHI analysis module loaded!")

## Compute IHI from Assessment Data

In [ ]:
# Load data
df = pd.read_csv('../data/examples/sample_assessment_data.csv')

# Compute IHI for each department
def compute_department_ihi(dept_df):
    process_integration = dept_df[dept_df['item_type'] == 'Process']['integration'].tolist()
    results_integration = dept_df[dept_df['item_type'] == 'Results']['integration'].tolist()
    
    if process_integration and results_integration:
        return compute_integration_health_index(process_integration, results_integration)
    return None

ihi_by_dept = []
for dept in df['department'].unique():
    dept_df = df[df['department'] == dept]
    ihi = compute_department_ihi(dept_df)
    
    if ihi is not None:
        ihi_by_dept.append({
            'Department': dept,
            'IHI': ihi,
            'Process_Integration_Mean': dept_df[dept_df['item_type'] == 'Process']['integration'].mean(),
            'Results_Integration_Mean': dept_df[dept_df['item_type'] == 'Results']['integration'].mean()
        })

ihi_df = pd.DataFrame(ihi_by_dept).sort_values('IHI', ascending=False)

print("\nDEPARTMENT INTEGRATION HEALTH INDEX")
print("=" * 80)
print(ihi_df.to_string(index=False))

## IHI Visualization: Current State

In [ ]:
# Create IHI visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart with color coding
colors_ihi = ['#27ae60' if ihi > 0.75 else '#f39c12' if ihi >= 0.60 else '#e74c3c' 
              for ihi in ihi_df['IHI']]

ax1.barh(ihi_df['Department'], ihi_df['IHI'], color=colors_ihi)
ax1.set_xlabel('Integration Health Index', fontsize=12)
ax1.set_ylabel('Department', fontsize=12)
ax1.set_title('Department Integration Health Index (IHI)', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 1.0)

# Add threshold lines
ax1.axvline(0.75, color='green', linestyle='--', linewidth=2, alpha=0.5, label='Strong Integration')
ax1.axvline(0.60, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='Moderate Threshold')
ax1.legend(fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Scatter: Process vs Results Integration
for idx, row in ihi_df.iterrows():
    color = '#27ae60' if row['IHI'] > 0.75 else '#f39c12' if row['IHI'] >= 0.60 else '#e74c3c'
    ax2.scatter(row['Process_Integration_Mean'], row['Results_Integration_Mean'],
               s=300, alpha=0.6, color=color, edgecolors='black', linewidth=2)
    ax2.annotate(row['Department'], 
                (row['Process_Integration_Mean'], row['Results_Integration_Mean']),
                fontsize=10, ha='center', va='bottom')

# Add diagonal line (perfect balance)
ax2.plot([0, 1], [0, 1], 'k--', alpha=0.3, linewidth=2, label='Perfect Balance')

ax2.set_xlabel('Mean Process Integration (ADLI)', fontsize=12)
ax2.set_ylabel('Mean Results Integration (LeTCI)', fontsize=12)
ax2.set_title('Process vs Results Integration Balance', fontsize=14, fontweight='bold')
ax2.set_xlim(0, 1.0)
ax2.set_ylim(0, 1.0)
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 12-Month IHI Trajectory Analysis

In [ ]:
# Load benchmark time-series data
benchmark_df = pd.read_csv('../data/examples/benchmark_results.csv')

departments = ['Computer Science', 'Business Admin', 'Engineering']
colors_dept = {'Computer Science': '#1f77b4', 'Business Admin': '#ff7f0e', 'Engineering': '#2ca02c'}

# Plot IHI trajectories
fig, ax = plt.subplots(figsize=(14, 8))

for dept in departments:
    dept_data = benchmark_df[benchmark_df['department'] == dept]
    
    ax.plot(dept_data['month'], dept_data['ihi_score'], 
           marker='o', linewidth=2.5, markersize=8,
           label=dept, color=colors_dept[dept])
    
    # Add trend line
    z = np.polyfit(dept_data['month'], dept_data['ihi_score'], 1)
    p = np.poly1d(z)
    ax.plot(dept_data['month'], p(dept_data['month']),
           linestyle='--', linewidth=1.5, alpha=0.5, color=colors_dept[dept])

# Add threshold bands
ax.axhspan(0.75, 1.0, alpha=0.1, color='green', label='Strong Integration Zone')
ax.axhspan(0.60, 0.75, alpha=0.1, color='yellow', label='Moderate Integration Zone')
ax.axhspan(0, 0.60, alpha=0.1, color='red', label='Weak Integration Zone')

ax.set_xlabel('Month', fontsize=13)
ax.set_ylabel('Integration Health Index (IHI)', fontsize=13)
ax.set_title('12-Month IHI Trajectory Analysis\n(Solid lines = actual, Dashed lines = trend)',
            fontsize=15, fontweight='bold')
ax.set_xlim(1, 12)
ax.set_ylim(0.5, 1.0)
ax.legend(fontsize=11, loc='lower right')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Compute growth rates
print("\n12-MONTH IHI GROWTH ANALYSIS")
print("=" * 60)
for dept in departments:
    dept_data = benchmark_df[benchmark_df['department'] == dept]
    initial = dept_data.iloc[0]['ihi_score']
    final = dept_data.iloc[-1]['ihi_score']
    growth = final - initial
    print(f"{dept:20} Initial: {initial:.3f}  Final: {final:.3f}  Growth: +{growth:.3f}")

## Silo Detection Analysis

In [ ]:
# Identify departments with silo risks (IHI < 0.60)
silo_analysis = benchmark_df.groupby('department').agg({
    'ihi_score': ['min', 'mean', 'max', 'std'],
    'organizational_score': 'mean'
}).round(3)

silo_analysis.columns = ['IHI_Min', 'IHI_Mean', 'IHI_Max', 'IHI_StdDev', 'Org_Score_Mean']
silo_analysis['Silo_Risk'] = silo_analysis['IHI_Min'] < 0.60
silo_analysis['Integration_Volatility'] = silo_analysis['IHI_StdDev'] > 0.10

print("\nSILO RISK ANALYSIS")
print("=" * 80)
print(silo_analysis)

# Visualize silo risk
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# IHI volatility
silo_analysis['IHI_StdDev'].plot(kind='barh', ax=ax1, color='#e74c3c')
ax1.set_xlabel('IHI Standard Deviation', fontsize=12)
ax1.set_ylabel('Department', fontsize=12)
ax1.set_title('Integration Volatility\n(Higher = More Unstable)', fontsize=13, fontweight='bold')
ax1.axvline(0.10, color='orange', linestyle='--', linewidth=2, label='High Volatility Threshold')
ax1.legend(fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# IHI vs Org Score correlation
for dept in silo_analysis.index:
    dept_data = benchmark_df[benchmark_df['department'] == dept]
    ax2.scatter(dept_data['ihi_score'], dept_data['organizational_score'],
               s=100, alpha=0.5, label=dept, color=colors_dept.get(dept, 'gray'))

# Fit overall trend
all_ihi = benchmark_df['ihi_score']
all_org = benchmark_df['organizational_score']
z = np.polyfit(all_ihi, all_org, 1)
p = np.poly1d(z)
ihi_range = np.linspace(all_ihi.min(), all_ihi.max(), 100)
ax2.plot(ihi_range, p(ihi_range), 'k--', linewidth=2, label=f'Trend: r={np.corrcoef(all_ihi, all_org)[0,1]:.3f}')

ax2.set_xlabel('Integration Health Index', fontsize=12)
ax2.set_ylabel('Organizational Score', fontsize=12)
ax2.set_title('IHI-Performance Correlation', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## IHI Improvement Velocity

In [ ]:
# Compute monthly IHI change rates
fig, ax = plt.subplots(figsize=(14, 7))

for dept in departments:
    dept_data = benchmark_df[benchmark_df['department'] == dept].copy()
    dept_data['ihi_change'] = dept_data['ihi_score'].diff()
    
    ax.plot(dept_data['month'].iloc[1:], dept_data['ihi_change'].iloc[1:],
           marker='o', linewidth=2, markersize=7,
           label=f"{dept} (Avg: {dept_data['ihi_change'].mean():.4f}/month)",
           color=colors_dept[dept])

ax.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.3)
ax.set_xlabel('Month', fontsize=13)
ax.set_ylabel('Monthly IHI Change', fontsize=13)
ax.set_title('IHI Improvement Velocity\n(Month-over-Month Change)', fontsize=15, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight: Positive IHI velocity indicates improving cross-category integration.")
print("Negative velocity signals deteriorating coordination requiring intervention.")

## Summary

### IHI Analysis Insights:

1. **Silo Detection**: IHI < 0.60 indicates weak cross-category integration requiring immediate attention
2. **Longitudinal Tracking**: 12-month trajectories reveal integration improvement or deterioration
3. **Performance Correlation**: Strong positive correlation (r ≈ 0.85-0.95) between IHI and organizational scores
4. **Volatility Monitoring**: High IHI standard deviation signals unstable integration patterns

### Typical Findings:
- **Low IHI + High Score**: Paradox indicating strong silos with local excellence
- **High IHI + Low Score**: Good integration but weak individual category performance
- **Increasing IHI Trajectory**: Successful cross-category collaboration initiatives

### Intervention Strategies:
1. **IHI < 0.60**: Establish cross-category coordination teams
2. **High Volatility**: Standardize integration mechanisms
3. **Declining Trajectory**: Conduct root cause analysis of silo formation

## Next Steps
- **Notebook 07**: Scalability benchmarks and performance testing
- **Notebook 08**: Publication-quality figures reproduction